In [ ]:
import numpy as np
import pandas as pd
from selenium import webdriver
import requests
import time
from selenium.webdriver.common.keys import Keys
import csv

In [ ]:
driver = webdriver.Chrome(executable_path ='C:/Users/91885/Desktop/glassdoor/chromedriver')
driver.maximize_window()

In [ ]:
df= pd.read_csv('company_list.csv')
company = df['coname'].tolist()
links = pd.read_csv('links.csv', header=None)[2].tolist()


In [ ]:
"""LOgging into the glassdoor website"""

emailid  = 'kshitizgupta108@gmail.com'
password = 'Tushar@123'


driver.get('https://www.glassdoor.co.in/Reviews/Glassdoor-Reviews-E100431.htm')
driver.find_element_by_xpath('//button[@class="gd-ui-button ml-std col-auto SearchStyles__newSearchButton css-1dqvyh7"]').click()


driver.find_element_by_partial_link_text('Sign In').click()
driver.find_element_by_id('userEmail').send_keys(emailid)
driver.find_element_by_id('userPassword').send_keys(password)
driver.find_element_by_xpath('//button[@class="gd-ui-button minWidthBtn css-8i7bc2"]').click()

In [ ]:
rating ={
    'SVGInline-svg css-hcqxoa-svg d-flex-svg':'yes',
    "SVGInline-svg css-10xv9lv-svg d-flex-svg":'not given',
    "SVGInline-svg css-1kiw93k-svg d-flex-svg":"no",
    "SVGInline-svg css-1h93d4v-svg d-flex-svg":"may be"
}
recomend = {
    'SVGInline-svg css-hcqxoa-svg d-flex-svg':'yes',
    "SVGInline-svg css-10xv9lv-svg d-flex-svg":'not given',
    "SVGInline-svg css-1kiw93k-svg d-flex-svg":"no",
    "SVGInline-svg css-1h93d4v-svg d-flex-svg":"may be"
}


rating_dir={
    "css-152xdkl":1,
    "css-19o85uz":2,
    "css-1ihykkv":3,
    "css-1c07csa":4,
    "css-1dc0bv4":5,
    "css-53ckbl":1.5,
    "css-3irkpa":3.5,
    "css-1dhpfkf":4.5,
    "css-xzfimx":2.5,
    "css-1nuumx7 e1hd5jg10":4,
    "css-vl2edp e1hd5jg10":3,
    "css-11rm5hs e1hd5jg10":3.5,
    "css-18v8tui e1hd5jg10":2,
    "css-xd4dom e1hd5jg10":1,
    "css-s88v13 e1hd5jg10":5,
    "css-qt3l8j e1hd5jg10":1.5,
    "css-z2gtf e1hd5jg10":2.5,
    "css-1areqgb e1hd5jg10":4.5
}
name_dir ={
    "Work/Life Balance":0,
    "Culture & Values":1,
    "Diversity & Inclusion":2,
    "Career Opportunities":3,
    "Compensation and Benefits":4,
    "Senior Management":5
}

In [ ]:
def get_reviews(driver, link):
    driver.get(link)
    
    if link.split('/')[3]=='Overview':
        driver.get_element_by_xpath('//div[@class="empLinks tbl "]/div/div/a').click()
        
    driver.find_element_by_xpath('//div[@class="search "]/div[2]/div[2]').click()
    driver.find_element_by_xpath('//div[@class="dropdownsContainer lib__EIFilterModuleStyles__desktopDropdowns  pt-sm pr-xl pb-std pl-md"]//div[@data-test="ContentFiltersSelectalocationDropdownContent"]/div').click()
    driver.find_element_by_xpath('//div[@class="dropdownsContainer lib__EIFilterModuleStyles__desktopDropdowns  pt-sm pr-xl pb-std pl-md"]//div[@data-test="ContentFiltersSelectalocationDropdownContent"]//input').send_keys('United States - All Cities')
    driver.find_element_by_xpath('//div[@class="dropdownsContainer lib__EIFilterModuleStyles__desktopDropdowns  pt-sm pr-xl pb-std pl-md"]//div[@data-test="ContentFiltersSelectalocationDropdownContent"]//input').send_keys(Keys.ENTER)
    link = driver.current_url

    total_reviews=driver.find_element_by_xpath('//div[@class="row d-flex justify-content-between "]/a[1]/span').text
    
    if total_reviews[-1]=='k':
        total_reviews = int(float(total_reviews[:-1])*1000)
    else:
        total_reviews = int(total_reviews)
    print(total_reviews)
    page = 0
    if(total_reviews%10==0):
        pages = int(total_reviews)//10
    else:
        pages = (total_reviews)//10 + 1
   


    print("Total pages",pages)
    for i in range(pages):
        modified_link = link
        if i!=0:
            modified_link = link.split('.htm?')[0]+'_IP'+str(i+1)+'.htm?'+link.split('.htm?')[-1]
        driver.get(modified_link)
        time.sleep(2)
        arr = driver.find_elements_by_xpath('//div[@class="mx-0"]')
        if(len(arr)==0):
            break
        for j in range(len(arr)):
            date = " ".join(driver.find_element_by_xpath(f'//li[{j+1}]/div//div[@class="mx-0"]/div[@class="px-std"][1]/span/span[@class="authorInfo"]/span[@class="authorJobTitle middle common__EiReviewDetailsStyle__newGrey"]').text.split()[:3])
            year = int(date.split()[-1])
          
            if year>=2008 and year<=2020:
                recomended = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"]/div/div/div[1]/span[1]/*[name()="svg"]').get_attribute('class')
                ceo_approval = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"]/div/div/div[2]/span[1]/*[name()="svg"]').get_attribute('class')
                business_outlook = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"]/div/div/div[3]/span[1]/*[name()="svg"]').get_attribute('class')
               
                if ceo_approval  in rating.keys():
                    ceo_approval = rating[ceo_approval]
                else:
                    ceo_approval = 'New'
                    
                if  recomended in recomend.keys():
                    recomended = recomend[recomended]
                else:
                    recomended = 'New'
                    
                if business_outlook  in recomend.keys():
                    business_outlook = recomend[business_outlook]
                else:
                    business_outlook = 'New'
                    
                    
                current_employee = False
                former_employee = False
                duration = "not given"
                history = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/span').text.split(',')
                if(len(history)):
                    if("Current Employee" in history[0]):
                        current_employee = True
                    else:
                        former_employee = True
                    if(len(history)>1):
                        duration = history[1]
           
                sub_rating = ["not given","not given","not given","not given","not given","not given"]
                
                
                if(len(driver.find_elements_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/aside'))):
                    element1= driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/aside')
                    driver.execute_script("arguments[0].click();",element1)
                    driver.execute_script("arguments[0].setAttribute('style','display:block;');",element1)
                    
                    len_new = len(driver.find_elements_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/aside/div/div/ul/li'))
                    for k in range(len_new):
                        subfield= driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/aside/div/div/ul/li[{k+1}]/div').text 
                        sub_rating[name_dir[subfield]] = rating_dir[driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/aside/div/div/ul/li[{k+1}]/div[2]').get_attribute('class')]
                    element1= driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/aside')
                    driver.execute_script("arguments[0].click();",element1)
                    driver.execute_script("arguments[0].setAttribute('style','display:block;');",element1)

                    
                    
                advice_to_management='not given'
                rating_unique = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="gdReview"]/div/div/div/div/span').text
                
                
                if(len(driver.find_elements_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][2]/div[@class="\n                        v2__EIReviewDetailsV2__continueReading \n                        v2__EIReviewDetailsV2__clickable \n                        v2__EIReviewDetailsV2__newUiCta mb \n                    "]'))):
                    driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][2]/div[@class="\n                        v2__EIReviewDetailsV2__continueReading \n                        v2__EIReviewDetailsV2__clickable \n                        v2__EIReviewDetailsV2__newUiCta mb \n                    "]').click()
                    if(len(driver.find_elements_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][2]/div[@class="v2__EIReviewDetailsV2__fullWidth "]'))>2):
                        advice_to_management = driver.find_elements_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][2]/div[@class="v2__EIReviewDetailsV2__fullWidth "][3]/p[2]')[0].text
                        
            
                pros_text = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][2]/div[1]/p[2]/span').text
                con_text = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][2]/div[2]/p[2]/span').text
                unique_heading = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][1]/h2/a').text
                
                
                te = driver.find_element_by_xpath(f'//li[{j+1}]/div//div[@class="mx-0"]/div[@class="px-std"][1]/span/span[@class="authorInfo"]/span[@class="authorJobTitle middle common__EiReviewDetailsStyle__newGrey"]').text.split('-')[-1]
                if(te==""):
                    te = 'not given'
                if len(driver.find_elements_by_xpath(f'//li[{j+1}]/div//div[@class="mx-0"]/div[@class="px-std"][1]/span/span[@class="authorInfo"]/span[@class="middle"]'))==0:
                    te2="not given"
                else:
                    te2 = driver.find_element_by_xpath(f'//li[{j+1}]/div//div[@class="mx-0"]/div[@class="px-std"][1]/span/span[@class="authorInfo"]/span[@class="middle"]').text[3:]
                unique_url = driver.find_element_by_xpath(f'//li[{j+1}]//div[@class="mx-0"]/div[@class="px-std"][1]/h2/a').get_attribute('href')
                review.append([date, te2,te,unique_heading,advice_to_management,rating_unique,sub_rating[0],sub_rating[1],sub_rating[2],sub_rating[3],sub_rating[4],sub_rating[5],ceo_approval,recomended,business_outlook,current_employee,former_employee, duration,con_text,pros_text, unique_url])
               
        print(f"{i} page completed")
    
    

In [ ]:
 review=[]

In [ ]:
def get_review_files(links, company):
    for i in range(1463,len(links)):
        #getting the list of reviews of the company
        
        print(company[i], links[i])
        get_reviews(driver, links[i])
        
        """Here we are creating a csv file for each company containing the reviews"""
        
        with open(f'./reviews/{company[i]}.csv', 'w',encoding ='utf8', newline="") as f:
                writer = csv.writer(f)
                writer.writerow(['ID number','Date','Location_USA_with_place_name','Position','Comment for company ',"advice to Management",'Overall ratting',"Work/Life Balance","Culture & Values","Diversity & Inclusion","Career Opportunities",'Compensation and Benefits','Senior Management','CEO Approval','Recommended',"Business Outlook","Current employee","Former employee",'Timeline',"cons","pros","review_url"])
                for j in range(len(review)):
                    writer.writerow([j+1]+review[j])
        print(f"link {i} completed")
        review.clear()


In [ ]:
get_review_files(links, company)